<a href="https://colab.research.google.com/github/t8li/Projects/blob/main/SUPPORT2_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install ucimlrepo

In [2]:
from ucimlrepo import fetch_ucirepo

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import datasets
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OneHotEncoder

In [4]:
# fetch dataset
support2 = fetch_ucirepo(id=880)

# metadata
print(support2.metadata)

# variable information
print(support2.variables)

df = pd.read_csv('https://archive.ics.uci.edu/static/public/880/data.csv')

df = df[['age','sex','dzgroup','scoma','race','sps','aps','diabetes','dementia','ca','meanbp','wblc','hrt','resp','temp','pafi','alb','bili','crea','sod','ph']]

{'uci_id': 880, 'name': 'SUPPORT2', 'repository_url': 'https://archive.ics.uci.edu/dataset/880/support2', 'data_url': 'https://archive.ics.uci.edu/static/public/880/data.csv', 'abstract': "This dataset comprises 9105 individual critically ill patients across 5 United States medical centers, accessioned throughout 1989-1991 and 1992-1994.\nEach row concerns hospitalized patient records who met the inclusion and exclusion criteria for nine disease categories: acute respiratory failure, chronic obstructive pulmonary disease, congestive heart failure, liver disease, coma, colon cancer, lung cancer, multiple organ system failure with malignancy, and multiple organ system failure with sepsis. The goal is to determine these patients' 2- and 6-month survival rates based on several physiologic, demographics, and disease severity information. \nIt is an important problem because it addresses the growing national concern over patients' loss of control near the end of life. It enables earlier deci

In [5]:
print("Number of observations:", len(df))

Number of observations: 9105


In [6]:
df.isnull().sum()

age            0
sex            0
dzgroup        0
scoma          1
race          42
sps            1
aps            1
diabetes       0
dementia       0
ca             0
meanbp         1
wblc         212
hrt            1
resp           1
temp           1
pafi        2325
alb         3372
bili        2601
crea          67
sod            1
ph          2284
dtype: int64

In [7]:
df.dzgroup.unique()

array(['Lung Cancer', 'Cirrhosis', 'ARF/MOSF w/Sepsis', 'Coma', 'CHF',
       'Colon Cancer', 'COPD', 'MOSF w/Malig'], dtype=object)

In [8]:
df['sex'].replace('female', 0, inplace=True)
df['sex'].replace('male', 1, inplace=True)

ohe = OneHotEncoder()
list1 = ['dzgroup', 'race', 'ca']
for i in list1:
  myohedzgroup = ohe.fit_transform(df[i].values.reshape(-1,1)).toarray()
  myohedzgroup=pd.DataFrame(myohedzgroup, columns=ohe.categories_[0])
  df=df.drop([i], axis=1)
  df=pd.concat([df,myohedzgroup],axis=1)

df = df.dropna(axis = 0, how = 'any')
df.drop(df.columns[31], axis=1)

,age,sex,scoma,sps,aps,diabetes,dementia,meanbp,wblc,hrt,...,Lung Cancer,MOSF w/Malig,asian,black,hispanic,other,white,metastatic,no,yes
0,62.84998,1,0.0,33.898438,20.0,0,0,97.0,6.000000,69.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
6,62.37097,1,0.0,17.296875,46.0,0,0,78.0,11.699219,120.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
17,63.66299,0,26.0,40.296875,58.0,0,0,69.0,30.097656,108.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
22,49.61298,0,0.0,31.699219,42.0,0,0,67.0,11.000000,120.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
25,53.84000,1,0.0,14.199219,16.0,0,0,134.0,7.799805,106.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9091,79.17596,1,9.0,22.097656,28.0,0,0,80.0,15.798828,107.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
9099,68.61597,0,0.0,15.298828,19.0,0,0,71.0,12.599609,110.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
9100,66.07300,1,0.0,16.296875,22.0,0,0,109.0,7.399414,104.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
9103,47.01999,1,0.0,40.195312,51.0,0,0,99.0,7.599609,110.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0


In [9]:
df.isnull().sum()

age                  0
sex                  0
scoma                0
sps                  0
aps                  0
diabetes             0
dementia             0
meanbp               0
wblc                 0
hrt                  0
resp                 0
temp                 0
pafi                 0
alb                  0
bili                 0
crea                 0
sod                  0
ph                   0
ARF/MOSF w/Sepsis    0
CHF                  0
COPD                 0
Cirrhosis            0
Colon Cancer         0
Coma                 0
Lung Cancer          0
MOSF w/Malig         0
asian                0
black                0
hispanic             0
other                0
white                0
NaN                  0
metastatic           0
no                   0
yes                  0
dtype: int64